In [1]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

ModuleNotFoundError: No module named 'sagemaker'

In [2]:
#role = sagemaker.get_execution_role()
role = 'arn:aws:iam::922607582968:role/service-role/AmazonSageMaker-ExecutionRole-20180531T213364'

In [3]:
#role = get_execution_role()
prefix = 'sagemakerDEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}
my_region = boto3.session.Session().region_name
#my_region='us-east-1'
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [4]:
bucket_name = 'narayani1'
s3 = boto3.resource('s3')


In [5]:
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ', e)

S3 bucket created successfully


In [6]:
try:
    model_data = pd.read_csv('./creditcard.csv',index_col=False)
    print('Success: Data loaded into dataframe')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe


In [7]:
model_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [8]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729),[int(0.7*len(model_data))])
#train_data1, train_data2 = np.split(train_data.sample(frac=1,random_state=1729),[int(0.40*len(train_data))])
test_data1, test_data2 = np.split(test_data.sample(frac=1,random_state=1729),[int(0.40*len(test_data))])
print(train_data.shape, test_data1.shape)

(199364, 31) (34177, 31)


In [9]:
test_df = pd.concat([train_data['Class'],train_data.drop(['Class'],axis=1)],axis=1)
test_df = test_df.reset_index()
test_df = test_df.drop(['Time','index'],axis=1)
test_df.shape

(199364, 30)

In [10]:
test_df.to_csv('train.csv',index=False,header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name,prefix),content_type='csv')

In [18]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[my_region],role,train_instance_count=1,train_instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name,prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [12]:
xgb.fit({'train':s3_input_train})

ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: No S3 objects found under S3 URL "s3://narayani1/sagemakerDEMO-xgboost-dm/train" given in input data source. Please ensure that the bucket exists in the selected region (us-east-1), that objects exist under that S3 prefix, and that the role "arn:aws:iam::922607582968:role/service-role/AmazonSageMaker-ExecutionRole-20180531T213364" has "s3:ListBucket" permissions on bucket "narayani1".

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

In [ ]:
test_data1a = test_data1.drop(['Class','Time'],axis=1)
test_data1a = test_data1a.reset_index()
test_data_array=test_data1a.drop(['index'],axis=1).as_matrix()
test_data1a.drop(['index'],axis=1).head()

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer=csv_serializer
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')
predictions_array=np.fromstring(predictions[1:],sep=',')
print(predictions_array.shape)

In [ ]:
cm = pd.crosstab(index=test_data1['Class'],columns=np.round(predictions_array),rownames=['Observed'],colnames=['Predicted'])
cm

In [ ]:
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.2f}%\n".format("Overall Classfication Rate: ",p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Fraud", "Fraud"))
print("Observed")
print("{0:<15}{1:<2.2f}% ({2:<}){3:>6.2f}% ({4:<})".format("No Fraud",tn/(tn+fn)*100,tn,fp/(tp+fp)*100,fp))
print("{0:<16}{1:<1.2f}% ({2:<}){3:>7.2f}% ({4:<}) \n".format("Fraud",fn/(tn+fn)*100,fn,tp/(tp+fp)*100,tp))

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()